# Time Series Modeling the S&P 500

### Disclaimer: This notebook should not be considered any kind of financial advice. It exists only for the purposes of practicing modeling and making predictions 

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.dummy import DummyClassifier 
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, recall_score, precision_score, f1_score, \
log_loss, auc, roc_auc_score, roc_curve, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost
from xgboost import XGBClassifier

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

import datetime as dt

# New import of my custom class
from classification_classes import Model, Importance

pd.set_option("display.max_rows", 25)
pd.set_option('display.max_columns', 35)
sns.set_style("dark")

# Preprocessing

In [32]:
# Load the S&P Data
sp = pd.read_csv('/Users/samalainabayeva/Desktop/Capstone Project/INDICES_FILLED.csv')

In [33]:
# Prepare the dataframe appropriately
sp.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
sp.set_index("Date", inplace=True)
sp = sp[sp['Index'] == "^GSPC"]

In [34]:
sp.shape

(15064, 24)

In [35]:
# Inspect Nulls
sp.isna().sum()

Index                     0
Adj Close                 0
Close                     0
High                      0
Low                       0
Open                      0
Volume                    0
garman_klass_vol          0
RSI                      20
lowest_bollinger_band    19
lower_bollinger_band     19
20_day_SMA               19
one_up_bollinger_band    19
upper_bollinger_band     19
ATR                      14
MACD                     25
dollar_volume(M)          0
Mkt-RF                   67
SMB                      67
HML                      67
RMW                      67
CMA                      67
GDP_Filled                0
Interest Rates            0
dtype: int64

In [36]:
# 92 total values out of 15064, so I will drop the nulls
print(df.shape[0] - df.dropna().shape[0])

# Create the permanent df object that we will work with
df = sp.dropna().copy()

92


### Working df object

In [50]:
# Have many -inf values due do zeros in the "open" feature being a denominator for garmin_klass, "zero-division"
# We will impute those values later when we model
df["garman_klass_vol"].replace(-np.inf, np.nan, inplace=True)

In [51]:
df.describe()

,Adj Close,Close,High,Low,Open,Volume,garman_klass_vol,RSI,lowest_bollinger_band,lower_bollinger_band,20_day_SMA,one_up_bollinger_band,upper_bollinger_band,ATR,MACD,dollar_volume(M),Mkt-RF,SMB,HML,RMW,CMA,GDP_Filled,Interest Rates
count,14972.000000,14972.000000,14972.000000,14972.000000,14972.000000,1.497200e+04,10421.000000,14972.000000,14972.000000,14972.000000,14972.000000,14972.000000,14972.000000,14972.000000,14972.000000,1.497200e+04,14972.000000,14972.000000,14972.000000,14972.000000,14972.000000,14972.000000,14972.000000
mean,933.231748,933.231748,938.680396,927.033412,903.256919,1.402298e+09,0.000074,53.487227,900.769995,915.584615,930.399236,945.213857,960.028477,12.309743,2.040333,2.806906e+06,0.000271,0.000078,0.000140,0.000138,0.000132,9064.084889,0.044720
std,1054.848850,1054.848850,1060.623892,1048.270047,1078.887655,1.828670e+09,0.000193,10.445511,1016.778954,1033.684447,1050.798942,1068.112392,1085.615278,16.526116,15.832952,4.823673e+06,0.010273,0.005445,0.005854,0.004004,0.003800,7236.276739,0.032613
min,62.279999,62.279999,63.230000,60.959999,0.000000,3.020000e+06,0.000000,15.561473,60.160341,63.756685,66.122499,67.775022,68.551043,0.869632,-237.020227,2.494290e+02,-0.174400,-0.111900,-0.050200,-0.030100,-0.058700,678.674000,-0.000500
25%,107.827501,107.827501,108.755001,106.907503,0.000000,3.072750e+07,0.000016,46.215390,104.856303,106.312262,107.773625,109.433707,110.977014,1.812673,-0.847751,3.090348e+03,-0.004200,-0.002800,-0.002400,-0.001700,-0.001800,2591.247000,0.016900
50%,453.059998,453.059998,454.220001,450.705002,453.050003,2.696550e+08,0.000033,54.048673,444.058837,447.628338,452.163000,456.907365,461.731549,4.220744,0.766159,1.216567e+05,0.000500,0.000200,0.000100,0.000100,0.000100,7115.652000,0.046800
75%,1325.182526,1325.182526,1332.974976,1313.934998,1324.837463,2.997040e+09,0.000072,60.985406,1281.393648,1300.849336,1322.913246,1345.605311,1369.072329,16.875214,5.312635,4.569764e+06,0.005100,0.003100,0.002500,0.001900,0.002000,14651.249000,0.061525
max,4796.560059,4796.560059,4818.620117,4780.040039,4804.509766,1.145623e+10,0.006982,84.072869,4610.997106,4653.915217,4716.387012,4795.193216,4925.005874,152.756467,92.583711,3.741963e+07,0.113500,0.061700,0.067300,0.045100,0.025300,27623.543000,0.171400


In [52]:
knn = KNNImputer(add_indicator=True)

In [55]:
ar = ARIMA(endog=df["Adj Close"], )

TypeError: ARIMA.__init__() missing 1 required positional argument: 'endog'